# Imports

In [1]:
import random
import pandas as pd
import numpy as np

from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, concatenate, Input, BatchNormalization
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Connect drive

In [2]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# Dataset

In [3]:
path_images="/content/drive/My Drive/uc project/images.npz"
dict_image_data = np.load(path_images)
image_data = dict_image_data['arr_0']

In [4]:
path_cities="/content/drive/My Drive/uc project/cities.npz"
dict_city_data = np.load(path_cities)
city_data = dict_city_data['arr_0']

path_class="/content/drive/My Drive/uc project/classes.npz"
dict_class_data = np.load(path_class)
class_data = dict_class_data['arr_0']

Set number of data to use from each city and class

In [5]:
n = 1000

In [6]:
city_df = pd.DataFrame(city_data, columns = ['city'])
minmax = {}
for city in list(city_df['city'].value_counts().index):
  minmax[city] = {
      'min': city_df[city_df['city'] == city].index[0],
      'max': city_df[city_df['city'] == city].index[-1]
  }
city_indices = np.concatenate([random.sample(range(mm['min'], mm['max']), n) for mm in minmax.values()])

In [7]:
class_df = pd.DataFrame(class_data, columns = ['class'])
minmax = {}
for label in list(class_df['class'].value_counts().index):
  minmax[label] = {
      'min': class_df[class_df['class'] == label].index[0],
      'max': class_df[class_df['class'] == label].index[-1]
  }
class_indices = np.concatenate([random.sample(range(mm['min'], mm['max']), n) for mm in minmax.values()])

Get subset of data

In [8]:
indices = np.concatenate([city_indices, class_indices])
# city_data = city_data[indices]
city_data = city_data
# class_data = class_data[indices]
class_data = class_data

In [9]:
# all 512 images resized to 128
# data_512 = image_data[:,0][indices]
data_512 = image_data[:,0]
# all 512 images cropped to 256 images and resized to 128
# data_256 = image_data[:,1][indices]
data_256 = image_data[:,1]
# all 512 images cropped to 128 images
# data_128 = image_data[:,2][indices]
data_128 = image_data[:,2]

In [10]:
city_data.shape, data_512.shape, data_512.shape

((28195,), (28195, 128, 128, 3), (28195, 128, 128, 3))

Tranforming labels to onehot encoding

In [37]:
from sklearn.preprocessing import LabelBinarizer
label_as_binary = LabelBinarizer()

city_labels = label_as_binary.fit_transform(city_data)
class_labels = label_as_binary.fit_transform(class_data)

Split train and test data


In [38]:
train_512, test_512, train_256, test_256, train_128, test_128, train_city, test_city, train_class, test_class = train_test_split(data_512, data_256, data_128, city_labels, class_labels, test_size=0.3, random_state=1)

# Network

In [13]:
def basic_model(input_name):
  # 1
  model_input = Input(shape=(128, 128, 3), name=input_name)
  conv2d1 = Conv2D(filters=32, kernel_size=(3,3), padding='same',activation='relu')(model_input)
  norm1 = BatchNormalization()(conv2d1)
  # 2
  pool1 = MaxPooling2D((2,2))(norm1)
  norm2 = BatchNormalization()(pool1)
  # 1
  conv2d2 = Conv2D(filters=32, kernel_size=(3,3),activation='relu')(norm2)
  norm3 = BatchNormalization()(conv2d2)
  # 2
  pool2 = MaxPooling2D((2,2))(norm3)
  norm4 = BatchNormalization()(pool2)
  # 15
  flat = Flatten()(norm4)
  dense = Dense(2048, activation='relu')(flat)
  model_output = Dropout(0.1)(dense)
  return model_input, model_output

## Basic model

Get input and initial output layers

In [39]:
basic_input, basic_output = basic_model('main_input')

Attach 2 outputs

In [40]:
#City
merged_hidden_city = Dense(2048, activation='softmax')(basic_output)
merged_drop_city = Dropout(0.1)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop_city)

#Class
merged_hidden_class = Dense(2048, activation='softmax')(basic_output)
merged_drop_class = Dropout(0.1)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop_class)

In [41]:
single_model = Model(inputs=basic_input, outputs=[city_output, class_output])
optimizer = keras.optimizers.Adam(learning_rate=0.1)
single_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [42]:
single_model.fit(train_512, {'city_output': train_city, 'class_output': train_class},
                 batch_size=32, epochs=20, shuffle=True,
                 validation_data=(test_512, {'city_output': test_city, 'class_output': test_class}))

Epoch 1/20
617/617 [==============================] - 24s 37ms/step - loss: 3.2648 - city_output_loss: 1.6046 - class_output_loss: 1.6601 - city_output_accuracy: 0.2646 - class_output_accuracy: 0.3413 - val_loss: 3.2547 - val_city_output_loss: 1.6067 - val_class_output_loss: 1.6480 - val_city_output_accuracy: 0.2728 - val_class_output_accuracy: 0.3046
Epoch 2/20
617/617 [==============================] - 22s 36ms/step - loss: 3.2801 - city_output_loss: 1.6053 - class_output_loss: 1.6748 - city_output_accuracy: 0.2663 - class_output_accuracy: 0.3248 - val_loss: 3.2736 - val_city_output_loss: 1.6236 - val_class_output_loss: 1.6500 - val_city_output_accuracy: 0.1934 - val_class_output_accuracy: 0.3046
Epoch 3/20
617/617 [==============================] - 22s 36ms/step - loss: 3.2778 - city_output_loss: 1.6054 - class_output_loss: 1.6724 - city_output_accuracy: 0.2636 - class_output_accuracy: 0.3156 - val_loss: 3.2671 - val_city_output_loss: 1.6133 - val_class_output_loss: 1.6538 - val_cit

## Final model

Three inputs
  1. e128
  1. e256
  1. e512

Two outputs
  1. city
  1. class

Create the three basic input models

In [18]:
e128_input, e128_output = basic_model('e128_input')
e256_input, e256_output = basic_model('e256_input')
e512_input, e512_output = basic_model('e512_input')

Concatenating outputs of three models

In [19]:
merged = concatenate([e128_output, e256_output, e512_output])

Defining the outputs

In [20]:
#City
merged_hidden_city = Dense(2048, activation='softmax')(merged)
merged_drop_city = Dropout(0.1)(merged_hidden_city)
city_output = Dense(len(np.unique(city_data)), activation='softmax', name='city_output')(merged_drop_city)

#Class
merged_hidden_class = Dense(2048, activation='softmax')(merged)
merged_drop_class = Dropout(0.1)(merged_hidden_class)
class_output = Dense(len(np.unique(class_data)), activation='softmax', name='class_output')(merged_drop_class)

In [21]:
triple_model = Model(inputs=[e128_input, e256_input, e512_input], outputs=[city_output, class_output])
optimizer = keras.optimizers.Adam(learning_rate=0.1)
triple_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

Training model

In [22]:
triple_model.fit({'e128_input': train_128, 'e256_input': train_256, 'e512_input': train_512}, {'city_output': train_city, 'class_output': train_class},
                 batch_size=32, epochs=20, shuffle=True,
                 validation_data=({'e128_input': test_128, 'e256_input': test_256, 'e512_input': test_512}, {'city_output': test_city, 'class_output': test_class}))

Epoch 1/20
617/617 [==============================] - 64s 100ms/step - loss: 3.6103 - city_output_loss: 1.6075 - class_output_loss: 2.0028 - city_output_accuracy: 0.2608 - class_output_accuracy: 0.3136 - val_loss: 3.5697 - val_city_output_loss: 1.6078 - val_class_output_loss: 1.9619 - val_city_output_accuracy: 0.1934 - val_class_output_accuracy: 0.3324
Epoch 2/20
617/617 [==============================] - 60s 98ms/step - loss: 3.5991 - city_output_loss: 1.6049 - class_output_loss: 1.9942 - city_output_accuracy: 0.2651 - class_output_accuracy: 0.3200 - val_loss: 3.5900 - val_city_output_loss: 1.6009 - val_class_output_loss: 1.9891 - val_city_output_accuracy: 0.2728 - val_class_output_accuracy: 0.3046
Epoch 3/20
617/617 [==============================] - 60s 98ms/step - loss: 3.6038 - city_output_loss: 1.6075 - class_output_loss: 1.9964 - city_output_accuracy: 0.2598 - class_output_accuracy: 0.3156 - val_loss: 3.5803 - val_city_output_loss: 1.6144 - val_class_output_loss: 1.9659 - val_ci